In [11]:
!pip install rasterio

In [12]:
!pip install folium

In [13]:
!pip install earthengine-api --upgrade

     |████████████████████████████████| 230 kB 11.1 MB/s eta 0:00:01
  Created wheel for earthengine-api: filename=earthengine_api-0.1.288-py3-none-any.whl size=260051 sha256=390d59a13cc18184fa7721de3596ea2ebe770a93f5480aaf66661d6166b48e30
  Stored in directory: /home/azureuser/.cache/pip/wheels/06/0b/85/edefcc40c4d90203465bdd1e2ee7990e38cb164ba9cf5172d2
Successfully built earthengine-api
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 0.1.287
    Uninstalling earthengine-api-0.1.287:
      Successfully uninstalled earthengine-api-0.1.287


In [14]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [15]:
import azureml.core
from azureml.core import Experiment, Workspace, Dataset, Datastore, ScriptRunConfig, Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
import os
import shutil
import glob
from os.path import join
import tensorflow as tf
#from tensorflow.python.keras import models
from sys import path
import numpy as np
import rasterio as rio
import json

# check core SDK version number

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.30.0


In [16]:
# make sure we have the most current version of github repo
%cd Satellite_ComputerVision
!git pull
%cd ..

/mnt/batch/tasks/shared/LS_root/mounts/clusters/mevans1/code/Users/mevans/Satellite_ComputerVision
Already up to date.
/mnt/batch/tasks/shared/LS_root/mounts/clusters/mevans1/code/Users/mevans


In [17]:
# attach our utilities folder to the path to import modules
path.append('/home/azureuser/cloudfiles/code/Users/mevans/Satellite_ComputerVision')

In [18]:
from utils.model_tools import get_model, make_confusion_matrix, weighted_bce
from utils.processing import get_training_dataset, normalize
from utils.prediction_tools import makePredDataset, make_array_predictions, get_img_bounds
from utils.clouds import basicQA

In [19]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
# get metadata about the workspace
print(ws.name, ws.location, ws.resource_group, sep='\t')
# list the registered datastores
ws.datastores

solar-compviz	eastus2	place


{'workspaceartifactstore': {
   "name": "workspaceartifactstore",
   "container_name": "azureml",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'solardatafilestore': {
   "name": "solardatafilestore",
   "container_name": "gee-data",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceblobstore': {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-20200d5f-4836-4a79-bf1c-7bd46e0d0c35",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-20200d5f-4836-4a79-bf1c-7bd46e0d0c35",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 }}

In [ ]:
# access our registered data share containing image data in this workspace
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatafilestore')

test_path = (datastore, 'CPK_solar/predict')

gee_train_dataset = Dataset.File.from_files(path = [gee_train_path])


In [ ]:
experiment_name = ''
exp = Experiment(workspace = ws, name = experiment_name)

In [ ]:
run_id = ''

run = ws.get_run(run_id)

run.tags

model_name = ''

In [ ]:
# if we haven't already registered our model
model = run.register_model(model_name= model_name,
                           tags=run.tags,
                           description = 'UNET model delineating anthropogenic change in pairs of Sentinel-2 imagery',
                           model_path='outputs/',
                           model_framework = 'Tensorflow',
                           model_framework_version= '2.0',
                           datasets = [('training', train_dataset), ('evaluation', eval_dataset), ('testing', test_dataset)])
print(model.name, model.id, model.version, sep='\t')

In [ ]:
model = Model(ws, model_name)
model_path = model.get_model_path(model_name, _workspace = ws)

In [ ]:
model_dir = Model.get_model_path(model_name)
weights_path = glob.glob(os.path.join(model_dir, '*.hdf5'))
model_path = glob.glob(os.path.join(model_dir, '*.h5'))
weights_path
model_path

In [ ]:
def get_weighted_bce(y_true,y_pred):
  return weighted_bce(y_true, y_pred, 1)
m = models.load_model(f'{model_path[0]}', custom_objects = {'get_weighted_bce': get_weighted_bce})
# m = get_model(depth = DEPTH, optim = OPTIMIZER, loss = get_weighted_bce, mets = METRICS, bias = None)
m.load_weights(f'{weights_path[0]}')

In [ ]:
# Create mountcontext and mount the dataset
aoi = 'WetlandMA'
test_path = (datastore, f'GEE/predict/{aoi}')
test_dataset = Dataset.File.from_files(path = [test_path])

test_mount = test_dataset.mount()

# open access to the mount point
test_mount.start() 

# Get the mount point
dataset_mount_folder = test_mount.mount_point
print(dataset_mount_folder)

testFiles = []

for root, dirs, files in os.walk(dataset_mount_folder):
    for f in files:
        testFiles.append(join(root, f))

In [ ]:
# Define some global variabes

BANDS = ['B2', 'B3', 'B4', 'B8', 'B2_1', 'B3_1', 'B4_1', 'B8_1']
RESPONSE = 'change'
FEATURES = BANDS + [RESPONSE]
print(FEATURES)
DEPTH = len(FEATURES)

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)

METRICS = {
    'logits':[tf.keras.metrics.MeanSquaredError(name='mse'), tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')],
    'classes':[tf.keras.metrics.MeanIoU(num_classes=2, name = 'mean_iou')]
    }

In [ ]:
predFiles = glob.glob(os.path.join(dataset_mount_folder, '*.gz'))
predFiles

In [ ]:
jsonFiles = [x for x in testFiles if '.json' in x]
jsonFiles
jsonFile = jsonFiles[0]

In [ ]:
predData = makePredDataset(predFiles, BANDS, one_hot = None)
iterator = iter(predData)
print(iterator.next())

In [ ]:
preds = make_array_predictions(imageDataset = predData, model = m, jsonFile = jsonFile)

In [ ]:
# We can visualize the predictions here in the notebook
from matplotlib import pyplot as plt
from matplotlib import colors

figure = plt.figure(figsize = (12,12))
# cmap = colors.ListedColormap(['#5dc5f1', '#50a886', '#3d6e1d', '#80e144', '#bafb85', '#d4a13e', "#e73522", "#9c9c9c", "#000000", "#706e22", "#e7e64b", "#fefc86", '#ad36ed'])
prob = preds[:, :, 0]
# cls = out_image[:, :, 0]

plt.imshow(prob)#, norm = colors.BoundaryNorm([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], cmap.N), cmap = cmap)

In [ ]:
# Folium setup.
import folium
print(folium.__version__)

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer